In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import retry

In [22]:
#Try selenium

In [23]:
df_games = pd.DataFrame(columns=['Title',
                                 'Link',
                                 'Description',
                                 'Geek Rating', 
                                 'Avg Rating',
                                 'Num Voters', 
                                 'List Price', 
                                 'Amazon Price'])

### API Parser

In [24]:
games_list = pd.read_csv('boardgames_ranks.csv')

In [25]:
games_list.iloc[0:2000]

,id,name,yearpublished,rank,bayesaverage,average,usersrated,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank
0,224517,Brass: Birmingham,2018,1,8.41581,8.59842,44835,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
1,161936,Pandemic Legacy: Season 1,2015,2,8.38155,8.52941,53143,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN
2,174430,Gloomhaven,2017,3,8.35758,8.59323,61748,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN
3,342942,Ark Nova,2021,4,8.33266,8.53712,42099,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.24384,8.60632,23407,NaN,NaN,NaN,NaN,NaN,5.0,3.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,345976,System Gateway (fan expansion for Android: Net...,2021,1996,6.20677,9.17895,563,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN
1996,320718,Hidden Leaders,2022,1997,6.20591,6.82511,2893,NaN,NaN,NaN,651.0,NaN,NaN,NaN,NaN
1997,155695,Age of War,2014,1998,6.20588,6.45122,7917,NaN,NaN,NaN,718.0,NaN,NaN,NaN,NaN
1998,236217,Timebomb,2016,1999,6.20548,7.15437,1705,NaN,NaN,NaN,NaN,98.0,NaN,NaN,NaN


In [26]:
df_interactions = pd.DataFrame(columns=['id','name','username','rating'])

In [27]:
def requests_retry_session(
                            retries=5, 
                            backoff_factor=0.3,
                            status_forcelist=(500, 502, 504),
                            session=None,
                            ):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

In [28]:
for i in games_list.iloc[0:2000]['id']:
    has_records = True
    #start from the 1st page
    page = 1
    while has_records:
        try:
            response = requests_retry_session().get(f'https://www.boardgamegeek.com/xmlapi/boardgame/{i}?comments=1&page={page}',
                                                     auth=('OlaviArg', 'I00magellan'),
                                                     timeout=15)
            if response.status_code == 200:
                page += 1
                #protect vs. more than 50000 comments for one items. Should rarely happen or never at all
                if page >= 500:
                    has_records = False
                soup = BeautifulSoup(response.text, 'xml')
                comments = soup.find_all('comment')
                if comments:
                    for comment in comments:
                        rating = comment.get('rating')
                        if rating != "N/A":
                            username = comment.get('username')
                            df_interactions.loc[len(df_interactions.index)] = [i,
                                                                               games_list.loc[games_list['id']==i,'name'].values[0],
                                                                               username,
                                                                               rating]
                else:
                    has_records = False
            time.sleep(5)
        except Exception as e:
            print(f"Error occurred: {e}")
            time.sleep(5)
            break

Error occurred: HTTPSConnectionPool(host='boardgamegeek.com', port=443): Read timed out.
Error occurred: HTTPSConnectionPool(host='boardgamegeek.com', port=443): Max retries exceeded with url: /xmlapi/boardgame/285967?comments=1&page=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='boardgamegeek.com', port=443): Read timed out. (read timeout=15)"))
Error occurred: HTTPSConnectionPool(host='boardgamegeek.com', port=443): Max retries exceeded with url: /xmlapi/boardgame/215?comments=1&page=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='boardgamegeek.com', port=443): Read timed out. (read timeout=15)"))
Error occurred: HTTPSConnectionPool(host='boardgamegeek.com', port=443): Max retries exceeded with url: /xmlapi/boardgame/293014?comments=1&page=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000024469E2C410>, 'Connection to boardgamegeek.com timed out. (connect timeout=15)'))
Error occurred: HTTPSConnectionPool(host='boardgamegeek.com',

In [29]:
df_interactions

,id,name,username,rating
0,224517,Brass: Birmingham,1bez,10
1,224517,Brass: Birmingham,2bit,7.5
2,224517,Brass: Birmingham,2d20,9
3,224517,Brass: Birmingham,35tango,10
4,224517,Brass: Birmingham,4characters,5
...,...,...,...,...
2414739,211716,John Company,zoedawg,10
2414740,211716,John Company,zonk67,9
2414741,211716,John Company,Zorzogoth,9
2414742,211716,John Company,Zowin11,8


In [30]:
df_interactions.to_csv('df_interactions.csv')

In [ ]:
df_interactions = pd.read_csv('df_interactions.csv')